In [ ]:
pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [ ]:
producer = KafkaProducer(bootstrap_servers=[':9092'], #give u r ip here
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))

In [ ]:
pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
pip install pyspark

In [ ]:
import spotipy
import pyspark
from spotipy.oauth2 import SpotifyClientCredentials
from pyspark.sql.functions import *

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id = "59ed7b9477804218bdc17dba795df9ec", client_secret = "bbf065f7ae5c4aeb8a6fe23e279521e9")

In [ ]:

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
playlist_link = "https://open.spotify.com/playlist/2LOxEzC4KmoWJ9NhW0kz5M"

In [ ]:
playlist_uri = playlist_link.split("/")[-1].split("?")[0]

In [ ]:
print(playlist_uri)

2LOxEzC4KmoWJ9NhW0kz5M


In [ ]:
data = sp.playlist_tracks(playlist_uri)

In [ ]:
print(data)

{'href': 'https://api.spotify.com/v1/playlists/2LOxEzC4KmoWJ9NhW0kz5M/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_at': '2017-04-18T02:56:20Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/maxaustralia'}, 'href': 'https://api.spotify.com/v1/users/maxaustralia', 'id': 'maxaustralia', 'type': 'user', 'uri': 'spotify:user:maxaustralia'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': None, 'available_markets': [], 'explicit': False, 'type': 'track', 'episode': False, 'track': True, 'album': {'available_markets': [], 'type': 'album', 'album_type': 'album', 'href': 'https://api.spotify.com/v1/albums/1TSZDcvlPtAnekTaItI3qO', 'id': '1TSZDcvlPtAnekTaItI3qO', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273a461aa96bd9a8fcd0a492aee', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02a461aa96bd9a8fcd0a492aee', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image

In [ ]:
data['items'][0]['track']['album']['id']

'1TSZDcvlPtAnekTaItI3qO'

In [ ]:
data['items'][0]['track']['album']['name']

'A Night At The Opera (2011 Remaster)'

In [ ]:
data['items'][0]['track']['album']['release_date']

'1975-11-21'

In [ ]:
data['items'][0]['track']['album']['total_tracks']

12

In [ ]:
data['items'][0]['track']['album']['external_urls']['spotify']

'https://open.spotify.com/album/1TSZDcvlPtAnekTaItI3qO'

In [ ]:
data['items'][0]['track']['album']['external_urls']

{'spotify': 'https://open.spotify.com/album/1TSZDcvlPtAnekTaItI3qO'}

In [ ]:
import pandas as pd

In [ ]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {'album_id': album_id,'album_name': album_name, 'release_date':album_release_date, 'total_tracks':album_total_tracks, 'url':album_url  }
    album_list.append(album_element)

In [ ]:
artist_list = []
for row in data['items']:
    for key,value in row.items():
        if key == 'track':
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'], 'artist_name':artist['name'],'external_url':artist['href']}
                artist_list.append(artist_dict)

In [ ]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_element = {'song_id':song_id , 'song_name':song_name, 'duration_ms':song_duration, 'url': song_url, 'popularity':song_popularity, 'song_added':song_added, 'album_id':album_id, 'artist_id':artist_id}
    song_list.append(song_element)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Spotifydata").getOrCreate()

In [ ]:
album_df=spark.createDataFrame(album_list)

In [ ]:
album_df.filter(length(col('release_date')) == 4).show()

+--------------------+--------------------+------------+------------+--------------------+
|            album_id|          album_name|release_date|total_tracks|                 url|
+--------------------+--------------------+------------+------------+--------------------+
|3SyxsGqgDR1LZFWU3...|   Friday On My Mind|        1965|          18|https://open.spot...|
|07Fr36M0hRPJrSJMF...|Grace (Legacy Edi...|        1994|          22|https://open.spot...|
|79x0PRGIZv33znrCk...|          No Secrets|        1972|          10|https://open.spot...|
|39EJWDJI8959h6aMZ...|       Human Frailty|        1986|          13|https://open.spot...|
|2iv4jmYWY966XXNbi...|For The Working C...|        1985|          12|https://open.spot...|
|2H371tT3IIvmqji0i...|20000 Watt RSL - ...|        1997|          18|https://open.spot...|
|6FCzvataOZh68j8OK...|London Calling (R...|        1979|          19|https://open.spot...|
|5QnHvl0ne6yEhRGw3...|Nobody Else (Deluxe)|        1995|          14|https://open.spot...|

In [ ]:
album_df = album_df.withColumn('release_date', when( (length(col('release_date')) == 4),concat(col('release_date'),lit('-01-01')) ).otherwise(col('release_date')))

In [ ]:
album_df.filter(length(col('release_date')) == 4).show()

+--------+----------+------------+------------+---+-----------+
|album_id|album_name|release_date|total_tracks|url|relese_date|
+--------+----------+------------+------------+---+-----------+
+--------+----------+------------+------------+---+-----------+



In [ ]:
album_df.schema

StructType([StructField('album_id', StringType(), True), StructField('album_name', StringType(), True), StructField('release_date', StringType(), True), StructField('total_tracks', LongType(), True), StructField('url', StringType(), True), StructField('relese_date', StringType(), True)])

In [ ]:
from pyspark.sql.types import StringType, DateType, FloatType

In [ ]:
album_df = album_df.withColumn('release_date',col('release_date').cast(DateType()))

In [ ]:
artist_df=spark.createDataFrame(artist_list)

In [ ]:
artist_df.show()

+--------------------+------------------+--------------------+
|           artist_id|       artist_name|        external_url|
+--------------------+------------------+--------------------+
|1dfeR4HaWDbWqFHLk...|             Queen|https://api.spoti...|
|3qm84nBOXUEQ2vnTf...|     Guns N' Roses|https://api.spoti...|
|6olE6TJLqED3rqDCT...|           Nirvana|https://api.spoti...|
|4x1nvY2FN8jxqAFA0...|       John Lennon|https://api.spoti...|
|1QxaPWG1POM8Ul6Ww...|      John Farnham|https://api.spoti...|
|36QJpDe2go2KgaRle...|      Led Zeppelin|https://api.spoti...|
|5a2EaR3hamoenG9rD...|            Prince|https://api.spoti...|
|711MCceyCBcFnzjGY...|             AC/DC|https://api.spoti...|
|3fMbdgg4jU18AjLCK...|   Michael Jackson|https://api.spoti...|
|22bE4uQ6baNwSHPVc...|The Rolling Stones|https://api.spoti...|
|0LcJLqbBmaGUft1e9...|              ABBA|https://api.spoti...|
|58lV9VcRSjABbAbfW...|          Bon Jovi|https://api.spoti...|
|1pJEZXU2hJApJW3rM...|     The Easybeats|https://api.sp

In [ ]:
songs_df = spark.createDataFrame(song_list)

In [ ]:
songs_df.show()

+--------------------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+
|            album_id|           artist_id|duration_ms|popularity|          song_added|             song_id|           song_name|                 url|
+--------------------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+
|1TSZDcvlPtAnekTaI...|1dfeR4HaWDbWqFHLk...|     354320|        21|2017-04-18T02:56:20Z|1AhDOtG9vPSOmsWgN...|Bohemian Rhapsody...|https://open.spot...|
|3I9Z1nDCL4E0cP62f...|3qm84nBOXUEQ2vnTf...|     354520|        29|2017-04-18T02:56:28Z|7o2CTH4ctstm8TNel...| Sweet Child O' Mine|https://open.spot...|
|2uEf3r9i2bnxwJQsx...|6olE6TJLqED3rqDCT...|     301920|        25|2017-04-18T02:56:41Z|1f3yAtsJtY87CTmM8...|Smells Like Teen ...|https://open.spot...|
|0xzaemKucrJpYhyl7...|4x1nvY2FN8jxqAFA0...|     187866|        75|2017-04-18T02:57:19Z|7pKfPom

In [ ]:
songs_df = songs_df.withColumn('song_added',col('song_added').cast(DateType()))

In [ ]:
songs_df.show()

+--------------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------------------+
|            album_id|           artist_id|duration_ms|popularity|song_added|             song_id|           song_name|                 url|
+--------------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------------------+
|1TSZDcvlPtAnekTaI...|1dfeR4HaWDbWqFHLk...|     354320|        21|2017-04-18|1AhDOtG9vPSOmsWgN...|Bohemian Rhapsody...|https://open.spot...|
|3I9Z1nDCL4E0cP62f...|3qm84nBOXUEQ2vnTf...|     354520|        29|2017-04-18|7o2CTH4ctstm8TNel...| Sweet Child O' Mine|https://open.spot...|
|2uEf3r9i2bnxwJQsx...|6olE6TJLqED3rqDCT...|     301920|        25|2017-04-18|1f3yAtsJtY87CTmM8...|Smells Like Teen ...|https://open.spot...|
|0xzaemKucrJpYhyl7...|4x1nvY2FN8jxqAFA0...|     187866|        75|2017-04-18|7pKfPomDEeI4TPT6E...|Imagine - Remaste...|https://open.spot...|
|2kzUFnhuzdPx

In [ ]:
album_df = album_df.dropDuplicates(subset = ['album_id'])

In [ ]:
artist_df = artist_df.dropDuplicates(subset = ['artist_id'])

In [ ]:
import requests
res = requests.get('https://api.spotify.com/v1/me/player/currently-playing')

In [ ]:
print(res)

<Response [401]>


In [ ]:
import requests
import time

from pprint import pprint


SPOTIFY_GET_CURRENT_TRACK_URL = 'https://api.spotify.com/v1/me/player/currently-playing'
ACCESS_TOKEN = '' #your access token


def get_current_track(access_token):
    response = requests.get(
        SPOTIFY_GET_CURRENT_TRACK_URL,
        headers={
            "Authorization": f"Bearer {access_token}"
        }
    )
    json_resp = response.json()

    track_id = json_resp['item']['id']
    track_name = json_resp['item']['name']
    artists = [artist for artist in json_resp['item']['artists']]

    link = json_resp['item']['external_urls']['spotify']

    artist_names = ', '.join([artist['name'] for artist in artists])

    current_track_info = {
    	"id": track_id,
    	"track_name": track_name,
    	"artists": artist_names,
    	"link": link
    }

    return current_track_info


def main():
	current_track_id = None
	while True:
	    current_track_info = get_current_track(ACCESS_TOKEN)

	    if current_track_info['id'] != current_track_id:
		    pprint(
		    	current_track_info,
		    	indent=4,
		    )
		    current_track_id = current_track_info['id']

	    time.sleep(1)


if __name__ == '__main__':
    main()

KeyError: 'item'